# 🎬 SmartClip AI - Google Colab

Transform long-form videos into viral-ready short clips using AI.

**Features:**
- 🤖 Multi-provider AI support (Groq FREE, Gemini, OpenAI)
- 🎙️ Fast transcription with Whisper
- 📝 Automatic word-level captions
- 🎯 Smart face tracking & active speaker detection
- 📺 Support for YouTube URLs & local files
- 💰 **100% FREE** with Groq (default)

---

## 📋 Instructions

1. **Run Cell 1** - Setup & Install dependencies
2. **Run Cell 2** - Enter your API key (Groq recommended - FREE!)
3. **Run Cell 3** - Upload video or enter YouTube URL
4. **Run Cell 4** - Configure options & generate clips
5. **Run Cell 5** - Download your clips

---

## 1️⃣ Setup & Installation

Run this cell first to install all dependencies.

In [ ]:
#@title 🔧 Install SmartClip AI
#@markdown This will clone the repository and install all dependencies.

import os
import subprocess

# Clone repository
if not os.path.exists('/content/sclip'):
    print("📥 Cloning SmartClip AI repository...")
    !git clone https://github.com/sakirsyarian/sclip.git /content/sclip
else:
    print("📁 Repository already exists, pulling latest changes...")
    !cd /content/sclip && git pull

# Change to project directory
os.chdir('/content/sclip')

# Install dependencies
print("\n📦 Installing dependencies...")
!pip install -q -r requirements.txt
!pip install -q -e .

# Update yt-dlp to latest version (fixes YouTube 403 errors)
print("\n🔄 Updating yt-dlp to latest version...")
!pip install -q -U yt-dlp

# Verify FFmpeg (pre-installed on Colab)
print("\n🔍 Checking FFmpeg...")
!ffmpeg -version | head -1

# Check yt-dlp version
print("\n🔍 Checking yt-dlp...")
!yt-dlp --version

# Check GPU availability
print("\n🖥️ Checking GPU...")
try:
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        print(f"✅ GPU available: {gpu_name}")
    else:
        print("⚠️ No GPU detected. Processing will use CPU.")
except:
    print("ℹ️ PyTorch not installed. Processing will use CPU.")

print("\n✅ Setup complete! Proceed to the next cell.")

## 2️⃣ Configure API Key

**Groq is FREE and recommended!** Get your key from [console.groq.com](https://console.groq.com)

Alternative: [Google AI Studio](https://aistudio.google.com/apikey) for Gemini

In [ ]:
#@title 🔑 Enter API Key
#@markdown **Groq (FREE)**: Get from [console.groq.com](https://console.groq.com)
#@markdown 
#@markdown **Gemini (Optional)**: Get from [aistudio.google.com](https://aistudio.google.com/apikey)

from getpass import getpass
import os

api_provider = "Groq (FREE - Recommended)" #@param ["Groq (FREE - Recommended)", "Gemini"]

# Try to load from Colab Secrets first
groq_key = None
gemini_key = None

try:
    from google.colab import userdata
    groq_key = userdata.get('GROQ_API_KEY')
    gemini_key = userdata.get('GEMINI_API_KEY')
    if groq_key:
        print("✅ Groq API key loaded from Colab Secrets")
    if gemini_key:
        print("✅ Gemini API key loaded from Colab Secrets")
except:
    pass

# Manual input if not found
if "Groq" in api_provider:
    if not groq_key:
        groq_key = getpass("Enter your Groq API key: ")
    if groq_key:
        os.environ['GROQ_API_KEY'] = groq_key
        masked = groq_key[:4] + "..." + groq_key[-4:]
        print(f"✅ Groq API key configured: {masked}")
        print("\n💡 Using Groq for transcription AND analysis (100% FREE!)")
    else:
        print("❌ No API key provided.")
else:
    if not gemini_key:
        gemini_key = getpass("Enter your Gemini API key: ")
    if gemini_key:
        os.environ['GEMINI_API_KEY'] = gemini_key
        masked = gemini_key[:4] + "..." + gemini_key[-4:]
        print(f"✅ Gemini API key configured: {masked}")
        print("\n⚠️ Note: You still need Groq API key for transcription (free)")
        if not groq_key:
            groq_key = getpass("Enter your Groq API key (for transcription): ")
            if groq_key:
                os.environ['GROQ_API_KEY'] = groq_key
                print("✅ Groq API key configured for transcription")
    else:
        print("❌ No API key provided.")

## 3️⃣ Upload Video

Choose one of the options below:
- **Option A**: Upload from your computer
- **Option B**: Use YouTube URL
- **Option C**: Use Google Drive

In [ ]:
#@title 📤 Upload Video
#@markdown Choose your video source:

video_source = "Upload from computer" #@param ["Upload from computer", "YouTube URL", "Google Drive"]
youtube_url = "" #@param {type:"string"}
drive_path = "/content/drive/MyDrive/video.mp4" #@param {type:"string"}

import os

video_path = None

if video_source == "Upload from computer":
    from google.colab import files
    print("📤 Please select a video file to upload...")
    uploaded = files.upload()
    if uploaded:
        filename = list(uploaded.keys())[0]
        video_path = f"/content/sclip/{filename}"
        os.rename(f"/content/{filename}" if os.path.exists(f"/content/{filename}") else filename, video_path)
        print(f"✅ Uploaded: {filename}")
    else:
        print("❌ No file uploaded")

elif video_source == "YouTube URL":
    if youtube_url:
        video_path = youtube_url
        print(f"✅ YouTube URL: {youtube_url}")
    else:
        print("❌ Please enter a YouTube URL in the field above")

elif video_source == "Google Drive":
    from google.colab import drive
    print("📁 Mounting Google Drive...")
    drive.mount('/content/drive')
    if os.path.exists(drive_path):
        video_path = drive_path
        print(f"✅ Found: {drive_path}")
    else:
        print(f"❌ File not found: {drive_path}")

if video_path:
    os.environ['SCLIP_VIDEO_PATH'] = video_path
    print(f"\n🎬 Video ready: {video_path}")

## 4️⃣ Generate Clips

Configure options and run SmartClip AI.

In [ ]:
#@title 🎬 Generate Clips
#@markdown Configure your clip settings:

max_clips = 5 #@param {type:"slider", min:1, max:10, step:1}
aspect_ratio = "9:16" #@param ["9:16", "1:1", "16:9"]
caption_style = "default" #@param ["default", "bold", "minimal", "karaoke"]
language = "id" #@param ["id", "en", "es", "fr", "de", "ja", "ko", "zh"]
min_duration = 45 #@param {type:"slider", min:15, max:60, step:5}
max_duration = 180 #@param {type:"slider", min:60, max:300, step:15}
analyzer = "groq" #@param ["groq", "gemini"]
no_captions = False #@param {type:"boolean"}
dry_run = False #@param {type:"boolean"}

import os

video_path = os.environ.get('SCLIP_VIDEO_PATH', '')

if not video_path:
    print("❌ No video selected. Please run Cell 3 first.")
else:
    os.chdir('/content/sclip')
    
    if video_path.startswith('http'):
        input_arg = f'-u "{video_path}"'
    else:
        input_arg = f'-i "{video_path}"'
    
    cmd = f"""sclip {input_arg} \
        -n {max_clips} \
        -a {aspect_ratio} \
        -s {caption_style} \
        -l {language} \
        --min-duration {min_duration} \
        --max-duration {max_duration} \
        --analyzer {analyzer} \
        -o /content/sclip/output \
        -f -v"""
    
    if no_captions:
        cmd += " --no-captions"
    if dry_run:
        cmd += " --dry-run"
    
    print("🚀 Running SmartClip AI...")
    print(f"📋 Provider: Groq (transcription) + {analyzer.capitalize()} (analysis)")
    print(f"\n📋 Command: {cmd}\n")
    print("=" * 60)
    
    !{cmd}
    
    print("\n" + "=" * 60)
    
    output_dir = '/content/sclip/output'
    if os.path.exists(output_dir):
        files = [f for f in os.listdir(output_dir) if f.endswith('.mp4')]
        if files:
            print(f"\n✅ Generated {len(files)} clip(s):")
            for f in files:
                size = os.path.getsize(os.path.join(output_dir, f)) / (1024*1024)
                print(f"   📹 {f} ({size:.1f} MB)")
        else:
            print("\n⚠️ No clips generated. Check the output above for errors.")
    else:
        print("\n⚠️ Output directory not found.")

## 5️⃣ Download Clips

Download your generated clips.

In [ ]:
#@title 📥 Download Clips
#@markdown Choose download method:

download_method = "Download to computer" #@param ["Download to computer", "Save to Google Drive", "Preview in notebook"]
drive_output_folder = "/content/drive/MyDrive/SmartClip_Output" #@param {type:"string"}

import os
import shutil

output_dir = '/content/sclip/output'

if not os.path.exists(output_dir):
    print("❌ No output directory found. Please run Cell 4 first.")
else:
    files = [f for f in os.listdir(output_dir) if f.endswith('.mp4')]
    
    if not files:
        print("❌ No clips found in output directory.")
    else:
        if download_method == "Download to computer":
            from google.colab import files as colab_files
            print(f"📥 Downloading {len(files)} clip(s)...")
            for f in files:
                filepath = os.path.join(output_dir, f)
                print(f"   Downloading: {f}")
                colab_files.download(filepath)
            print("\n✅ Download complete!")
            
        elif download_method == "Save to Google Drive":
            from google.colab import drive
            drive.mount('/content/drive', force_remount=False)
            os.makedirs(drive_output_folder, exist_ok=True)
            
            print(f"📁 Copying {len(files)} clip(s) to Google Drive...")
            for f in files:
                src = os.path.join(output_dir, f)
                dst = os.path.join(drive_output_folder, f)
                shutil.copy2(src, dst)
                print(f"   ✅ {f}")
            
            txt_files = [f for f in os.listdir(output_dir) if f.endswith('.txt')]
            for f in txt_files:
                src = os.path.join(output_dir, f)
                dst = os.path.join(drive_output_folder, f)
                shutil.copy2(src, dst)
            
            print(f"\n✅ Saved to: {drive_output_folder}")
            
        elif download_method == "Preview in notebook":
            from IPython.display import HTML, display
            import base64
            
            print(f"🎬 Previewing {len(files)} clip(s)...\n")
            
            for f in files:
                filepath = os.path.join(output_dir, f)
                with open(filepath, 'rb') as video_file:
                    video_data = base64.b64encode(video_file.read()).decode('utf-8')
                
                print(f"📹 {f}")
                display(HTML(f'''
                <video width="300" controls>
                    <source src="data:video/mp4;base64,{video_data}" type="video/mp4">
                </video>
                '''))
                print()

## 🧹 Cleanup (Optional)

Remove temporary files to free up space.

In [ ]:
#@title 🧹 Cleanup
#@markdown Remove temporary files and free up space.

delete_output = False #@param {type:"boolean"}
delete_uploaded_video = False #@param {type:"boolean"}
delete_repository = False #@param {type:"boolean"}

import os
import shutil

if delete_output:
    output_dir = '/content/sclip/output'
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
        print("✅ Deleted output directory")
    else:
        print("ℹ️ Output directory not found")

if delete_uploaded_video:
    video_path = os.environ.get('SCLIP_VIDEO_PATH', '')
    if video_path and os.path.exists(video_path) and not video_path.startswith('http'):
        os.remove(video_path)
        print(f"✅ Deleted: {video_path}")
    else:
        print("ℹ️ No uploaded video to delete")

if delete_repository:
    os.chdir('/content')
    if os.path.exists('/content/sclip'):
        shutil.rmtree('/content/sclip')
        print("✅ Deleted repository")
    else:
        print("ℹ️ Repository not found")

if not any([delete_output, delete_uploaded_video, delete_repository]):
    print("ℹ️ No cleanup options selected.")

---

## 📚 Tips & Troubleshooting

### API Keys
- **Groq (FREE)**: Get from [console.groq.com](https://console.groq.com) - Recommended!
- **Gemini**: Get from [aistudio.google.com](https://aistudio.google.com/apikey)

### YouTube Download Error (403 Forbidden)
- Re-run Cell 1 to update yt-dlp
- Or upload video manually instead

### Rate Limits (Error 429)
- Groq: 30 requests/minute (free tier)
- Gemini: 15 requests/minute (free tier)
- Wait a few minutes and try again

### Video Requirements
- Minimum duration: 60 seconds
- Supported formats: MP4, MKV, AVI, MOV, WebM

### Links
- [GitHub Repository](https://github.com/sakirsyarian/sclip)
- [Groq Console](https://console.groq.com)
- [Report Issues](https://github.com/sakirsyarian/sclip/issues)

---

Made with ❤️ by SmartClip AI